In [ ]:
!pip install roboflow
!pip install ultralytics

In [9]:
# Импорт библиотек
from roboflow import Roboflow
from ultralytics import YOLO
import json

In [ ]:
# Получение и подготовка датасета для обучения модели
rf = Roboflow(api_key="091mcn4Yvn3gWaa9FCeq")
project = rf.workspace("parkinglot-64itp").project("parking_lot-7nsmw")
version = project.version(19)
dataset = version.download("yolov8")

In [ ]:
# Загружаем предобученную модель для обучения 
model = YOLO("yolov8n.pt")

# Тренировка
model.train(data="Parking_Lot-19/data.yaml", epochs=20, imgsz=640)  # У меня ушло полтора часа

In [3]:
# Загрузка твоих весов
model = YOLO("runs/detect/train9/weights/best.pt")

# Тест на одной картинке
results = model.predict("6.png", save=True, imgsz=640, show_labels=False, show_conf=False)


image 1/1 C:\Users\1\_JupyterNotebook_content\_ParkingML\6.png: 480x640 57 emptylots, 4 moving cars, 14 nonemptylots, 105.3ms
Speed: 7.6ms preprocess, 105.3ms inference, 9.5ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\detect\predict10


In [4]:
model = YOLO("runs/detect/train9/weights/best.pt")

results = model.predict(
    source="6.png",
    imgsz=640,
    save=True,             # сохраняет изображение с боксами
    save_json=True,        # сохраняет JSON в формате COCO
    show_labels=False,
    show_conf=False,
    project="runs/detect", # куда сохранять
    name="predict_coco"    # подпапка для этого прогона
)



image 1/1 C:\Users\1\_JupyterNotebook_content\_ParkingML\6.png: 480x640 57 emptylots, 4 moving cars, 14 nonemptylots, 52.1ms
Speed: 3.8ms preprocess, 52.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\detect\predict_coco


In [8]:
# 1. Загружаем модель
model = YOLO("runs/detect/train9/weights/best.pt")

# 2. Предсказываем на одном изображении
results = model.predict("6.png", imgsz=640, save=False)[0]

# 3. Считываем размеры исходного изображения
img = cv2.imread("6.png")
height, width = img.shape[:2]

# 4. Готовим структуру COCO
coco = {
    "images": [
        {"id": 1, "file_name": "6.png", "width": width, "height": height}
    ],
    "annotations": [],
    "categories": []
}

# 5. Добавляем категории 
for cls_id, cls_name in model.names.items():
    coco["categories"].append({"id": cls_id, "name": cls_name})

# 6. Пробегаем по всем боксам и формируем аннотации
ann_id = 1
for box in results.boxes:
    x1, y1, x2, y2 = box.xyxy[0].tolist()
    w, h = x2 - x1, y2 - y1
    cat = int(box.cls[0].item()) 
    score = float(box.conf[0].item())

    # if 
    coco["annotations"].append({
        "id": ann_id,
        "image_id": 1,
        "category_id": cat,
        "bbox": [x1, y1, w, h],
        "area": w * h,
        "iscrowd": 0,
        "score": score
    })
    ann_id += 1

# 7. Сохраняем JSON на диск
with open("predictions_coco.json", "w", encoding="utf-8") as f:
    json.dump(coco, f, ensure_ascii=False, indent=2)



image 1/1 C:\Users\1\_JupyterNotebook_content\_ParkingML\6.png: 480x640 57 emptylots, 4 moving cars, 14 nonemptylots, 38.8ms
Speed: 2.8ms preprocess, 38.8ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
